In [12]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Define the path to the data directory
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')

# Create the folder if it doesn't exist
os.makedirs(data_dir, exist_ok=True)

# Build the full path to the file
file_path = os.path.join(data_dir, 'Melbourne.csv')

# Load the dataset
df = pd.read_csv(file_path)

# Replace all empty values with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Count how many duplicate rows exist
print(df.duplicated().sum())

# Remove all duplicate rows
df = df.drop_duplicates()

# Optional: reset the index after dropping duplicates
df = df.reset_index(drop=True)

1


In [13]:
# Function to check for missing values
def check_missing_columns(df):
    missing_cols = [col for col in df.columns if df[col].isnull().any()]
    return missing_cols

# Initial check for missing values
cols_with_missing = check_missing_columns(df)
print("Columns with missing values:", cols_with_missing)

Columns with missing values: ['Price', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude', 'Longtitude', 'Regionname', 'Propertycount']


In [14]:
# Step 1: Fill grouping categorical columns first (Suburb, Regionname, CouncilArea)
# Assume Suburb might have misses (even if not), fill with mode by Postcode (if available) or global mode
df['Suburb'] = df['Suburb'].fillna(df.groupby('Postcode')['Suburb'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
df['Suburb'].fillna(df['Suburb'].mode()[0], inplace=True)

# Fill Regionname with mode by Suburb, then global mode
df['Regionname'] = df['Regionname'].fillna(df.groupby('Suburb')['Regionname'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
df['Regionname'].fillna(df['Regionname'].mode()[0], inplace=True)

# Fill CouncilArea with mode by Suburb, then by Regionname
df['CouncilArea'] = df['CouncilArea'].fillna(df.groupby('Suburb')['CouncilArea'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
df['CouncilArea'] = df['CouncilArea'].fillna(df.groupby('Regionname')['CouncilArea'].transform(lambda x: x.mode()[0] if not x.mode().empty else np.nan))
df['CouncilArea'].fillna(df['CouncilArea'].mode()[0], inplace=True)

C:\Users\Kamal Mustafayev\AppData\Local\Temp\ipykernel_5556\82594779.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Suburb'].fillna(df['Suburb'].mode()[0], inplace=True)
C:\Users\Kamal Mustafayev\AppData\Local\Temp\ipykernel_5556\82594779.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always be

In [15]:
# Step 2: Fill numerical columns based on groupings
# Price: mean by Suburb, fallback to Regionname
df['Price'] = df['Price'].fillna(df.groupby('Suburb')['Price'].transform(lambda x: round(x.mean(), 1)))
df['Price'] = df['Price'].fillna(df.groupby('Regionname')['Price'].transform(lambda x: round(x.mean(), 1)))

# 1️ Fill by Suburb (mode preferred, fallback to median)
df['Propertycount'] = df.groupby('Suburb')['Propertycount'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.median())
)
# 2️ Fill remaining NaN by Regionname (mode preferred, fallback to median)
df['Propertycount'] = df.groupby('Regionname')['Propertycount'].transform(
    lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else x.median())
)
# 3️ Final fallback (in case there’s still something left)
df['Propertycount'].fillna(df['Propertycount'].median(), inplace=True)

# YearBuilt: median global (no strong grouping dependency)
df['YearBuilt'].fillna(df['YearBuilt'].median(), inplace=True)

# Bedroom2: mean global, or by Rooms if needed
df['Bedroom2'] = df['Bedroom2'].fillna(df['Rooms'])

# Other means: Bathroom, Car
columns_to_replace_mean = ['Bathroom', 'Car']
for column in columns_to_replace_mean:
    df[column].fillna(round(df[column].mean(), 1), inplace=True)

c:\Users\Kamal Mustafayev\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:1214: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\Kamal Mustafayev\AppData\Local\Temp\ipykernel_5556\3789620152.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Propertycount'].fillna(df['Propertycount'].median(), inplace=True)
C:\Users\Kamal Mustafayev\AppData\Local\Temp\ipykernel_5556\3789620152.py:18: FutureWarning: A value is trying to be set o

In [16]:
# Step 3: Handle zeros (replace invalid zeros with NaN and fill)
zero_columns_to_replace = ['Distance', 'Landsize', 'BuildingArea', 'Postcode']
for col in zero_columns_to_replace:
    df[col] = df[col].replace(0, np.nan)
    df[col] = df[col].fillna(df.groupby('Suburb')[col].transform('median'))
    df[col] = df[col].fillna(df.groupby('Regionname')[col].transform('median'))

# Bathroom zeros: median by Rooms
df['Bathroom'] = df['Bathroom'].replace(0, np.nan)
df['Bathroom'] = df['Bathroom'].fillna(df.groupby('Rooms')['Bathroom'].transform('median'))

# Bedroom2 zeros: sync with Rooms
df.loc[df['Bedroom2'] == 0, 'Bedroom2'] = df.loc[df['Bedroom2'] == 0, 'Rooms']

In [17]:
# Step 4: Drop unnecessary columns
df.drop(columns=['Lattitude', 'Longtitude'], inplace=True)

# Rename Rooms to Bedroom (after all Rooms-based fills)
df = df.rename(columns={'Rooms': 'Bedroom'})

In [18]:
# Step 5: Fix invalid values
# YearBuilt corrections
replacements = {1196: 1996, 1800: 1900, 1830: 1930, 1850: 1950}
df['YearBuilt'] = df['YearBuilt'].replace(replacements)
mask = df['YearBuilt'] == 2106
df.loc[mask, 'YearBuilt'] = df.loc[mask, 'Date'].str[-4:].astype(int)

In [19]:
# Step 6: Final checks
# Check for remaining missing values
has_missing = df.isnull().values.any()
print("All missing values gone?", not has_missing)

# Function to check string lengths
def check_string_lengths(series, min_length=4):
    return [val for val in series.unique() if isinstance(val, str) and len(val) < min_length]

string_cols = df.select_dtypes(include='object').columns.tolist()
for col in string_cols:
    print(f"{col} short values: {check_string_lengths(df[col])}")

# Check dates
def check_date_format(series, date_format="%d-%m-%Y"):
    invalid = []
    for val in series:
        try:
            datetime.strptime(val, date_format)
        except (ValueError, TypeError):
            invalid.append(val)
    return invalid

print("Invalid dates:", check_date_format(df['Date']))

# Check invalid YearBuilt, Price, Postcode, etc.
invalid_years = df[(df['YearBuilt'] <= 1850) | (df['YearBuilt'] > 2025)]['YearBuilt'].unique()
print("Invalid YearBuilt:", invalid_years)

invalid_prices = df[df['Price'] <= 18000]['Price'].unique()
print("Invalid Prices:", invalid_prices)

is_valid_postcode = (df['Postcode'].astype(str).str.len() == 4) & (df['Postcode'].astype(str).str.isdigit())
invalid_postcodes = df[~is_valid_postcode]['Postcode'].unique()
print("Invalid Postcodes:", invalid_postcodes)

# Check zeros in numerics
def check_zero_columns(df):
    zero_cols = [col for col in df.columns if pd.api.types.is_numeric_dtype(df[col]) and (df[col] == 0).any()]
    return zero_cols

print("Columns with zeros:", check_zero_columns(df))

# Replace all values in 'Bedroom2' that are 20 or 30 with 12
df['Bedroom2'] = df['Bedroom2'].replace([20, 30], 12)

All missing values gone? True
Suburb short values: ['Kew']
Address short values: []
Type short values: ['h', 'u', 't']
Method short values: ['SS', 'S', 'VB', 'SP', 'PI', 'SN', 'W', 'PN', 'SA']
SellerG short values: ['Kay', 'Ray', 'RT', 'Jas', 'FN', 'Tim', 'HAR', 'RW', 'R&H', 'One', 'C21', 'GL', 'YPA', 'J', 'Del', 'ASL', 'Re', 'RE', 'Ham', 'Vic', 'Joe', 'Red', 'LJ', 'Win', 'New', 'MSM', 'S&L', 'Ken', 'JRW', 'Oak', 'Jim', 'Max', 'L', 'Ash', 'LLC', 'Ace', 'M.J', 'U', 'JY', 'Le', 'The', 'MJ', 'SN', 'P', 'PRD', 'T']
Date short values: []
CouncilArea short values: []
Regionname short values: []
Invalid dates: []
Invalid YearBuilt: []
Invalid Prices: []
Invalid Postcodes: [3067. 3042. 3206. 3078. 3018. 3025. 3143. 3032. 3147. 3034. 3183. 3103.
 3104. 3204. 3165. 3128. 3019. 3186. 3187. 3056. 3055. 3105. 3125. 3124.
 3126. 3054. 3163. 3162. 3161. 3148. 3068. 3058. 3066. 3108. 3084. 3185.
 3184. 3040. 3041. 3065. 3031. 3011. 3146. 3046. 3043. 3188. 3122. 3081.
 3166. 3079. 3021. 3033. 3101. 310

In [20]:
# Step 7: Type conversions
columns_to_int = ['YearBuilt', 'Postcode', 'Bedroom', 'Bedroom2', 'Car', 'Bathroom','Propertycount']
for col in columns_to_int:
    df[col] = df[col].astype(int)

In [21]:
# Build output file path
output_path = os.path.join(data_dir, 'Melbourne_cleaned.csv')

# Save cleaned dataset
df.to_csv(output_path, index=False)
print(f"✅ Cleaned dataset saved to: {output_path}")
print(df.head())

✅ Cleaned dataset saved to: c:\Users\Kamal Mustafayev\Desktop\melbourne\melbourne-housing-analysis\data\Melbourne_cleaned.csv
       Suburb             Address  Bedroom Type      Price Method SellerG  \
0  Abbotsford       68 Studley St        2    h  1026500.0     SS  Jellis   
1  Abbotsford        85 Turner St        2    h  1480000.0      S  Biggin   
2  Abbotsford     25 Bloomburg St        2    h  1035000.0      S  Biggin   
3  Abbotsford  18/659 Victoria St        3    u  1026500.0     VB  Rounds   
4  Abbotsford        5 Charles St        3    h  1465000.0     SP  Biggin   

         Date  Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  \
0  03-09-2016       2.5      3067         2         1    1     126.0   
1  03-12-2016       2.5      3067         2         1    1     202.0   
2  04-02-2016       2.5      3067         2         1    0     156.0   
3  04-02-2016       2.5      3067         3         2    1     181.5   
4  04-03-2017       2.5      3067         3        